# A Fuzzy Model with rating fading for Managing Natural Noise in RSs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

from time import time
from datetime import datetime
from numpy import save, load
from collections import defaultdict

from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise import KNNBaseline
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

tic = time()
np.random.seed(1593523459)

### Datasets:
* MovieLens-100K
* MovieLens-1M

### Variables:
* ```PU``` - dictionary with user profiles [ userID, fuzzyTransformation( profiles ) ]
* ```PI``` - dictionary with item profiles [ itemID, fuzzyTransformation( profiles ) ]
* ```variable``` - profiles with [ 0, 0, 0 ]
* ```TRESHOLD``` - treshold for detecting noise rating

In [2]:
PU = defaultdict(list)
PI = defaultdict(list)
variable = np.array([0, 0, 0])

TRIGGER = 'M'

if TRIGGER == 'K':
    TRESHOLD_1 = 1.0
    TRESHOLD_2 = 1.7
    ml_name = 'ml-100k'
    data = load('../datasets/main/train_set_100K.npy')
#     data = np.loadtxt('../datasets/main/ml-100k.data', skiprows=0, delimiter='\t').astype('int32')

elif TRIGGER == 'M':
    TRESHOLD_1 = 1.0
    TRESHOLD_2 = 1.7
    ml_name = 'ml-1m'
    data = load('../datasets/main/train_set_1M.npy')
#     data = np.loadtxt('../datasets/main/ml-1M.dat', skiprows=0, delimiter='::').astype('int32')

In [3]:
data

array([[        1,         1,         5, 978824268],
       [        1,        48,         5, 978824351],
       [        1,       150,         5, 978301777],
       ...,
       [     6040,      3703,         4, 964828575],
       [     6040,      3751,         4, 964828782],
       [     6040,      3819,         5, 963272166]])

In [4]:
# Convert to DataFrame and filter the dataset by 'user' and 'item'
data_csv = pd.DataFrame(data, columns = ['user', 'item', 'rating', 'timestamp'])
data_csv.sort_values(['user', 'item'], ascending=[True, True], inplace=True)
data_csv.reset_index(inplace=True)
data_csv.drop('index', axis=1, inplace=True)
data_csv

,user,item,rating,timestamp
0,1,1,5,978824268
1,1,48,5,978824351
2,1,150,5,978301777
3,1,260,4,978300760
4,1,527,5,978824195
...,...,...,...,...
900184,6040,3671,4,997454367
900185,6040,3683,4,960971696
900186,6040,3703,4,964828575
900187,6040,3751,4,964828782


In [5]:
users = np.unique(data[:, 0]).tolist() # list of unique users
items = np.unique(data[:, 1]).tolist() # list of unique items

In [6]:
n_u = len(users)     # number of users
n_m = len(items)     # number of movies
n_r = data.shape[0]  # number of ratings

print("USERS: {}\t ITEMS: {}\t RATINGS: {}".format(n_u, n_m, n_r))

USERS: 6040	 ITEMS: 3694	 RATINGS: 900189


In [7]:
# Make dict for users and movies, where udict[u_id] = index(0..942), and mdict[m_id] = index(0..1681)
udict = {}
for i, u_id in enumerate(users):
    udict[u_id] = i
    
mdict = {}
for i, m_id in enumerate(items):
    mdict[m_id] = i

In [8]:
# Empty matrix
matrix = np.zeros((n_u, n_m), dtype='int32')

In [9]:
# Fill matrix
for i in range(n_r):
    u_id = data[i, 0]
    m_id = data[i, 1]
    r = data[i, 2]
    
    matrix[udict[u_id], mdict[m_id]] = int(r)

In [10]:
matrix

array([[5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0]])

# Fading part

In [11]:
min_timestamp = min(data_csv['timestamp'])
max_timestamp = max(data_csv['timestamp'])

timestamp = datetime.fromtimestamp(min_timestamp)
print(timestamp.strftime('%Y-%m-%d %H:%M:%S'))

timestamp = datetime.fromtimestamp(max_timestamp)
print(timestamp.strftime('%Y-%m-%d %H:%M:%S'))

2000-04-26 08:05:32
2003-03-01 02:49:50


In [12]:
# Movielens 1M
# 2000-04-26 08:05:32
# 2003-03-01 02:49:50

# Movielens 25M
# 1995-01-09 20:46:49
# 2019-11-21 18:15:03

### Timestamp Preprocessing

In [13]:
PERCENTILES = [.1, .2, .3, .4, .5, .6, .7, .8, .9]

In [14]:
ts_thresholds = data_csv['timestamp'].describe(percentiles=PERCENTILES).astype('int32').tolist()[3:]

In [15]:
# [min, 10%, 20%, 30%, 40%, 50%, 60%, 70%, 80%, 90%, max]
for i, ts in enumerate(ts_thresholds):
    print(f'{i}\t{ts}')

0	956703932
1	960682425
2	964152976
3	965678170
4	967588706
5	973018024
6	974688074
7	974862060
8	975767742
9	978132500
10	1046454590


In [16]:
def normolised(ts):
    return 1 - ( ts - min(ts_thresholds) ) / ( max(ts_thresholds[:-2]) - min(ts_thresholds) ) # normalised from 0% to 80%

In [17]:
def get_timestamp(u_id, i_id):
    return data_csv[(data_csv['user'] == u_id) & (data_csv['item'] == i_id)]['timestamp'].iloc[0]

In [18]:
def get_fading_weight(ts):
    if ts <= ts_thresholds[8]:
        return normolised(ts)
    else:
        return 0.0

In [19]:
def apply_fading(r, w):
    new_rating = abs(r - w)
    if new_rating < 1.0:
        return 1.0
    else:
        return new_rating

In [20]:
for i, ts in enumerate(ts_thresholds):
    print(f'{i}\t{normolised(ts)}')

0	1.0
1	0.7913065121819824
2	0.6092573310371852
3	0.5292526520144714
4	0.4290346997793201
5	0.14423758944303366
6	0.056634429319217916
7	0.04750792207853516
8	0.0
9	-0.12404435419782289
10	-3.707907705752418


### Deep Learning Model: Sparse FC

In [21]:
# Load full dataset
full_data = np.loadtxt('../datasets/main/ml-1M.dat', skiprows=0, delimiter='::').astype('int32')

# these dictionaries define a mapping from user/movie id to to user/movie number (contiguous from zero)
full_udict = {}
for i, u in enumerate(np.unique(full_data[:, 0]).tolist()):
    full_udict[u] = i
full_mdict = {}
for i, m in enumerate(np.unique(full_data[:, 1]).tolist()):
    full_mdict[m] = i

# Load predicted matrix
deep_learn_matrix_load = load('../datasets/main/predicted_matrix_1M.npy')

In [22]:
print(np.unique(full_data[:, 0]).shape[0])  # number of users
print(np.unique(full_data[:, 1]).shape[0])  # number of movies
print(full_data.shape[0])  # number of ratings
print(deep_learn_matrix_load.shape)

6040
3706
1000209
(6040, 3706)


### Rest

In [23]:
def predict_rating(u_id, i_id):
    rating = round(deep_learn_matrix_load[full_udict[u_id], full_mdict[i_id]], 3)  # round(, 3)

    return rating

In [24]:
def fuzzy_transformation(r):
    if 1 <= r <= 1.5:
        low = 1
    elif 1.5 < r <= 2:
        low = -2*r + 4
    elif 2 < r <= 5:
        low = 0
        
    if 1 <= r <= 1.5:
        med = 0
    elif 1.5 < r <= 2:
        med = 2*r - 3
    elif 2 < r <= 3:
        med = 1
    elif 3 < r <= 4:
        med = -r + 4
    elif 4 < r <= 5:
        med = 0
        
    if 1 <= r <= 3:
        high = 0
    elif 3 < r <= 4:
        high = r - 3
    elif 4 < r <= 5:
        high = 1

    return np.array([low, med, high])

In [25]:
def f1(array):
    updated = []
    
    for x in array:
        if 0 <= x <= 0.35:
            updated.append(0)
        elif 0.35 < x <= 1:
            updated.append(20/13*x - 7/13)
    
    return np.array(updated)

In [26]:
def dist(a, b):
    return sum(abs(a - b))

In [27]:
def skip_unsuitable_profiles(pu, pi):
    if (pu == variable).all() or (pi == variable).all() or (dist(pu, pi) > TRESHOLD_1):
        return True

In [28]:
def skip_unnoisy_ratings(pu, pi, r):
    if min( dist(pu, fuzzy_transformation(r)), dist(pi, fuzzy_transformation(r)) ) < TRESHOLD_2:
        return True

In [29]:
# def dissimilarity(p, r):
#     return dist(p, r) - 1

def dissimilarity(p, r):
    return (dist(p, r) - TRESHOLD_2) / (2 - TRESHOLD_2)

In [30]:
def get_noise_degree(pu, pi, r):
    return min(dissimilarity(pu, fuzzy_transformation(r)), dissimilarity(pi, fuzzy_transformation(r)))

In [31]:
def get_profiles(users, items):
    # User Profile
    for u_id in users:  
        count = 0
        ft = np.array([0, 0, 0])

        for i_id in items:
            r = matrix[udict[u_id], mdict[i_id]]
            if r == 0: continue
            count += 1
            ft += fuzzy_transformation(r)
            
        PU[u_id] = f1(ft / count) 
            
    # Item Profile    
    for i_id in items:
        count = 0
        ft = np.array([0, 0, 0])
        
        for u_id in users:
            r = matrix[udict[u_id], mdict[i_id]]
            if r == 0: continue
            count += 1
            ft += fuzzy_transformation(r)
        
        PI[i_id] = f1(ft / count)
        
    return PU, PI

In [33]:
def noise_correction(matrix, PU, PI):
    final_matrix = matrix.copy().astype('float32')
    count_time = 0
    
    for u_id in users:
        for i_id in items:
            
            # Extract current rating!!!
            r = matrix[udict[u_id], mdict[i_id]]
            if r == 0: continue
                
            # Skip the unsuitable profiles and not noisy ratings!!!
            if skip_unsuitable_profiles(PU[u_id], PI[i_id]): continue
            if skip_unnoisy_ratings(PU[u_id], PI[i_id], r): continue

            # If rating is outdated then reduce rating!!!
            timestamp = get_timestamp(u_id, i_id)
            fading_weight = get_fading_weight(timestamp)
            fading_r = apply_fading(r, fading_weight)

            # If rating is not outdated then use weighted prediction!!!
            n = predict_rating(u_id, i_id)
            noise_degree = get_noise_degree(PU[u_id], PI[i_id], r)
            newRating = r * (1 - noise_degree) + n * noise_degree
            
            # Update matrix!!!
            if fading_weight == 0:
                final_matrix[udict[u_id], mdict[i_id]] = newRating
            else:
                final_matrix[udict[u_id], mdict[i_id]] = fading_r
                
            count_time += 1
            if count_time % 100 == 0:
                print('{0}\t{1}\t{2:0.3f}\t{3:0.3f}\t{4:0.3f}\t{5:0.3f}'.format(count_time, r, n, noise_degree, newRating, fading_r) )
            
#             final_matrix[udict[u_id], mdict[i_id]] = newRating
#             final_matrix[udict[u_id], mdict[i_id]] = n
            
    print('\nTotal:', count_time)
    
    return final_matrix

In [34]:
def reshape_data(final_matrix):
    data_sorted = np.array(data_csv)                                             # get sorted data
    new_ratings = final_matrix.reshape(-1, 1)                                    # reshape into one column
    new_ratings = new_ratings[new_ratings != 0].reshape(-1, 1).astype('float32') # remove empty ratings

    new_data = np.hstack((data_sorted[:,:2], new_ratings))                       # combine
    
    return new_data

# Running

In [35]:
# Get matrix with degree of noises
PU, PI = get_profiles(users, items)

In [ ]:
# Correct the Matrix
final_matrix = noise_correction(matrix, PU, PI)
#     r        n      n_d      r*     f_r

100	3	4.070	0.258	3.277	3.000
200	3	3.446	0.231	3.103	3.000
300	3	3.618	0.194	3.120	3.000
400	3	3.377	0.263	3.099	3.000
500	1	1.992	0.031	1.031	1.000
600	4	3.693	0.172	3.947	4.000
700	3	4.070	0.201	3.215	3.000
800	3	3.879	0.057	3.050	3.000
900	3	3.713	0.342	3.244	3.000
1000	3	3.736	0.551	3.406	3.000
1100	3	3.661	0.249	3.165	3.000
1200	1	2.222	0.193	1.236	1.000
1300	2	2.610	0.109	2.067	2.000
1400	3	3.634	0.343	3.218	3.000
1500	2	3.980	0.072	2.143	2.000
1600	3	4.031	0.091	3.094	3.000
1700	3	4.277	0.435	3.555	3.000
1800	3	3.429	0.303	3.130	3.000
1900	1	3.009	0.523	2.050	1.000
2000	4	3.047	0.123	3.883	4.000
2100	4	2.953	0.101	3.894	4.000
2200	3	3.478	0.406	3.194	3.000
2300	3	2.649	0.142	2.950	3.000
2400	3	3.237	0.037	3.009	3.000
2500	2	3.113	0.069	2.077	2.000
2600	3	3.128	0.357	3.046	3.000
2700	1	3.617	0.210	1.551	1.000
2800	3	2.958	0.405	2.983	3.000
2900	3	2.772	0.122	2.972	3.000
3000	1	0.832	0.146	0.975	1.000
3100	1	2.122	0.157	1.176	1.000
3200	3	3.034	0.157	3.005	3.000
3300	3	3.445	0.10

26100	1	2.146	0.377	1.432	1.000
26200	3	2.914	0.084	2.993	2.966
26300	3	3.617	0.000	3.000	2.965
26400	3	4.145	0.410	3.470	2.965
26500	3	3.542	0.307	3.167	2.965
26600	2	3.147	0.131	2.150	2.000
26700	3	3.726	0.487	3.353	2.963
26800	2	3.778	0.463	2.824	1.962
26900	4	2.680	0.134	3.823	3.962
27000	1	1.621	0.237	1.147	1.000
27100	1	1.998	0.224	1.224	1.000
27200	1	1.109	0.237	1.026	1.000
27300	2	3.498	0.038	2.057	2.000
27400	3	3.220	0.308	3.068	2.965
27500	3	3.059	0.022	3.001	3.000
27600	3	3.386	0.023	3.009	2.960
27700	1	0.855	0.132	0.981	1.000
27800	1	1.860	0.189	1.163	1.000
27900	3	3.253	0.015	3.004	2.960
28000	1	2.871	0.189	1.354	1.000
28100	3	3.939	0.043	3.040	2.960
28200	1	1.943	0.285	1.269	1.000
28300	2	2.889	0.018	2.016	1.960
28400	1	2.546	0.481	1.743	1.000
28500	3	3.728	0.438	3.319	2.993
28600	4	2.643	0.044	3.941	3.958
28700	2	3.489	0.309	2.461	1.965
28800	1	1.277	0.335	1.093	1.000
28900	3	4.316	0.354	3.466	2.957
29000	3	3.452	0.366	3.166	2.957
29100	3	4.152	0.687	3.791	2.963
29200	3	

51800	3	3.274	0.151	3.041	2.944
51900	3	3.561	0.128	3.072	2.945
52000	3	3.603	0.088	3.053	3.000
52100	3	3.877	0.504	3.442	2.944
52200	3	3.408	0.056	3.023	2.944
52300	1	1.678	0.292	1.198	1.000
52400	3	3.495	0.575	3.285	2.944
52500	3	2.958	0.006	3.000	3.000
52600	3	3.577	0.006	3.003	2.945
52700	2	2.837	0.148	2.124	1.944
52800	2	3.973	0.517	3.019	1.944
52900	3	3.202	0.033	3.007	3.000
53000	3	3.068	0.158	3.011	2.949
53100	4	3.354	0.201	3.870	3.944
53200	3	3.516	0.302	3.156	2.980
53300	1	1.223	0.347	1.077	1.000
53400	3	3.596	0.441	3.263	2.944
53500	3	3.892	0.194	3.173	3.000
53600	3	3.575	0.073	3.042	2.944
53700	1	2.286	0.540	1.694	1.000
53800	3	3.583	0.357	3.208	2.944
53900	1	1.685	0.298	1.204	1.000
54000	3	4.657	0.562	3.930	2.944
54100	3	3.427	0.346	3.148	3.000
54200	3	3.747	0.118	3.088	3.000
54300	3	3.230	0.149	3.034	2.946
54400	3	3.183	0.485	3.089	2.944
54500	3	3.910	0.593	3.539	2.944
54600	3	2.905	0.409	2.961	2.944
54700	3	3.857	0.178	3.152	2.989
54800	3	2.933	0.134	2.991	3.000
54900	3	

In [38]:

new_data = reshape_data(final_matrix)

In [39]:
# # Save final matrix

# if TRIGGER == 'K':
#     file_matrix = '../datasets/final_filter/final_matrix_100K_{0}.npy'.format(TRESHOLD)
#     file_new_data = '../datasets/final_filter/dataset_100K_{0}.npy'.format(TRESHOLD)
    
#     save(file_new_data, new_data)
#     save(file_matrix, final_matrix)

#     new_data_load = load(file_new_data)
    
# elif TRIGGER == 'M':
#     file_matrix = '../datasets/final_filter/final_matrix_1M_{0}F_W.npy'.format(TRESHOLD)
#     file_new_data = '../datasets/final_filter/dataset_1M_{0}F_W.npy'.format(TRESHOLD)
    
#     save(file_new_data, new_data)
#     save(file_matrix, final_matrix)

#     new_data_load = load(file_new_data)

### Results

In [ ]:
(new_data == new_data_load).all()

In [ ]:
new_data_csv = pd.DataFrame(new_data_load, columns = ['user', 'item', 'rating'])
new_data_csv

In [ ]:
print('Preprocessing time: {0} mins!'.format( int( (time() - tic) / 60.0 ) ) )

# Evaluate accuracy

In [ ]:
TRESHOLD

In [ ]:
# new_data_load = load('../datasets/final_filter/dataset_100K_{0}.npy'.format(TRESHOLD))
new_data_load = load('../datasets/final_filter/dataset_1M_{0}.npy'.format(TRESHOLD))
new_data_csv2 = pd.DataFrame(new_data_load, columns = ['user', 'item', 'rating'])
new_data_csv2

In [ ]:
# # A reader is still needed but only the rating_scale param is requiered.
# reader = Reader(rating_scale=(1, 5))

# # The columns must correspond to user id, item id and ratings (in that order).
# dataset_2 = Dataset.load_from_df(new_data_csv2[['user', 'item', 'rating']], reader)

# # Build an algorithm, and train it.
# knn_2 = KNNBaseline(k=60)

# # Run 5-fold cross-validation and print results.
# cross_validate(knn_2, dataset_2, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
# # A reader is still needed but only the rating_scale param is requiered.
# reader = Reader(rating_scale=(1, 5))

# # The columns must correspond to user id, item id and ratings (in that order).
# dataset_2 = Dataset.load_from_df(new_data_csv[['user', 'item', 'rating']], reader)

# # Retrieve the trainset.
# trainset, testset = train_test_split(dataset_2, test_size=.1)

# # Build an algorithm, and train it.
# knn_2 = KNNBaseline(k=60)

# # Fit data
# knn_2.fit(trainset)

# # Test
# knn_predictions = knn_2.test(testset)

# # Then compute RMSE
# accuracy.rmse(knn_predictions)

# # get a prediction for specific users and items.
# knn_pred = knn_2.predict(str(196), str(302), r_ui=4, verbose=True)

In [ ]:
# RMSE: 0.8926     Base                    1.5 <= TRESHOLD <= 1.8          Base
# RMSE: 0.8053     TRESHOLD = 1
# RMSE: 0.8731     TRESHOLD = 1.5

# RMSE: 0.8856     TRESHOLD = 1.6

# RMSE: 0.8874     TRESHOLD = 1.7
# RMSE: 0.8917     TRESHOLD = 1.8

In [ ]:
# RMSE: 0.8926     Base                    1.5 <= TRESHOLD <= 1.8          Train_data
# RMSE: 0.8061     TRESHOLD = 1
# RMSE: 0.8786     TRESHOLD = 1.5

# RMSE: 0.8896     TRESHOLD = 1.6
# RMSE: 0.8942     TRESHOLD = 1.7

# RMSE: 0.8949     TRESHOLD = 1.8

### Visualization

In [ ]:
new_data_csv['updated'] = new_data_csv['rating'].round(1)
new_data_csv

In [ ]:
data_csv['rating'].value_counts().sort_index(ascending=False)

In [ ]:
new_data_csv['updated'].value_counts().sort_index(ascending=False)

In [ ]:
# Create a Figure with Axes
fig1, ax = plt.subplots()

# Coordinates 
data_rating = data_csv['rating'].value_counts().sort_index(ascending=False)

x1 = data_rating.index
y1 = data_rating.values

# Draw the bars and add the text
ax.bar(x1, y1, width=0.1, zorder=2)

# Update the yticks
locs, labels = plt.yticks()
locs_updated = ['%dk'%(round(v/1000)) if v!=0 else '0' for v in locs]
plt.yticks(ticks=locs[:-1], labels=locs_updated[:-1])

# Title and X, Y Labels
ax.set_xlabel('Rating')
ax.set_ylabel('Count')
ax.set_title('Distribution of 1M ratings')

# Some additional functions
ax.grid(zorder=0)
fig1.tight_layout()

# Save and show figure
# fig1.savefig('./datasets/images/1. ratings bar.pdf')
fig1.show()

In [ ]:
# Create a Figure with Axes
fig2, ax = plt.subplots()

# Coordinates 
new_data_rating = new_data_csv['updated'].value_counts().sort_index(ascending=False)

x2 = new_data_rating.index
y2 = new_data_rating.values

# Draw the bars and add the text
ax.bar(x2, y2, width=0.1, zorder=2)

# Update the yticks
locs, labels = plt.yticks()
locs_updated = ['%dk'%(round(v/1000)) if v!=0 else '0' for v in locs]
plt.yticks(ticks=locs[:-1], labels=locs_updated[:-1])

# Title and X, Y Labels
ax.set_xlabel('Rating')
ax.set_ylabel('Count')
ax.set_title('Distribution of 1M ratings')

# Some additional functions
ax.grid(zorder=0)
fig2.tight_layout()

# Save and show figure
# fig1.savefig('./datasets/images/1. ratings bar.pdf')
fig2.show()

### Functions for Getting Deprocation and Updating Ratings

In [ ]:
# def timestamp_fuzzy_transformation(ts):
#     if ts_desc[0] <= ts <= ts_desc[3]:
#         low = ts_desc[10]
#     elif ts_desc[3] < ts <= ts_desc[4]:
#         low = -ts + ts_desc[4]
#     elif ts_desc[4] < ts <= ts_desc[10]:
#         low = ts_desc[0]
        
#     if ts_desc[0] <= ts <= ts_desc[3]:
#         med = ts_desc[0]
#     elif ts_desc[3] < ts <= ts_desc[4]:
#         med = -ts + ts_desc[4]
#     elif ts_desc[4] < ts <= ts_desc[6]:
#         med = ts_desc[10]
#     elif ts_desc[6] < ts <= ts_desc[7]:
#         med = -ts + ts_desc[7]
#     elif ts_desc[7] < ts <= ts_desc[10]:
#         med = ts_desc[0]
        
#     if ts_desc[0] <= ts <= ts_desc[6]:
#         high = ts_desc[0]
#     elif ts_desc[6] < ts <= ts_desc[7]:
#         high = -ts + ts_desc[7]
#     elif ts_desc[7] < ts <= ts_desc[10]:
#         high = ts_desc[10]

#     return np.array([low, med, high])


# def get_fading_b(ts):
#     if ts <= ts_desc[1]:
#         return normolised(ts)             # 1.0-0.8 (1.00-0.79)
#     elif ts_desc[1] < ts <= ts_desc[2]:
#         return normolised(ts)             # 0.8-0.6 (0.79-0.60)
#     elif ts_desc[2] < ts <= ts_desc[3]:
#         return normolised(ts)             # 0.6-0.4 (0.60-0.52)
#     elif ts_desc[3] < ts <= ts_desc[4]:
#         return normolised(ts)             # 0.4-0.2 (0.52-0.42)
#     elif ts_desc[4] < ts <= ts_desc[5]:
#         return normolised(ts)             # 0.2-0.1 (0.42-0.14)
#     elif ts_desc[5] < ts <= ts_desc[6]:
#         return normolised(ts)             # 0.1-0.0 (0.14-0.05)
#     elif ts_desc[6] < ts <= ts_desc[7]:
#         return normolised(ts)             # 0.1-0.0 (0.05-0.04)
#     elif ts_desc[7] < ts <= ts_desc[8]:
#         return normolised(ts)             # 0.1-0.0 (0.04-0.00)
#     elif ts > ts_desc[8]:
#         return 0